In [1]:
import os
os.chdir('F:\\nanopore_data\\JUfosmid\\seperation\\')

In [2]:
os.getcwd()

'F:\\nanopore_data\\JUfosmid\\seperation'

In [56]:
%%bash
du -hsc trim/* |grep fa

113M	trim/trim_barcode01.fa
189M	trim/trim_barcode02.fa
27M	trim/trim_barcode03.fa
133M	trim/trim_barcode04.fa
225M	trim/trim_barcode05.fa
214M	trim/trim_barcode06.fa
156M	trim/trim_barcode07.fa
110M	trim/trim_barcode08.fa
158M	trim/trim_barcode09.fa
116M	trim/trim_barcode10.fa
236M	trim/trim_barcode11.fa
112M	trim/trim_barcode12.fa
71M	trim/trim_barcode13.fa
204M	trim/trim_barcode14.fa
110M	trim/trim_barcode15.fa
246M	trim/trim_barcode16.fa
154M	trim/trim_barcode17.fa
168M	trim/trim_barcode18.fa
176M	trim/trim_barcode19.fa
99M	trim/trim_barcode20.fa
122M	trim/trim_barcode21.fa
158M	trim/trim_barcode22.fa
178M	trim/trim_barcode23.fa
211M	trim/trim_barcode24.fa
141M	trim/trim_barcode25.fa
217M	trim/trim_barcode26.fa
48M	trim/trim_barcode27.fa
280M	trim/trim_barcode28.fa
291M	trim/trim_barcode29.fa
351M	trim/trim_barcode30.fa
271M	trim/trim_barcode31.fa
237M	trim/trim_barcode32.fa
186M	trim/trim_barcode33.fa
300M	trim/trim_barcode34.fa
268M	trim/trim_barcode35.fa
164M	trim/trim_barcode36

In [3]:
import pandas as pd
from Bio import SeqIO

In [4]:
def fileName(j=3):
    if j <10:
        i = "barcode0"+str(j)
    else:
        i = "barcode"+str(j)
    return(i)


In [6]:
barcoder=fileName(3)

In [13]:
triminfo_fh = str("blastn_backbone_info_filtered/%s.csv" % barcoder)
fasta_fh = str("read_with_backbone/%s.fa" % barcoder)
output_fh = str("read_trimmed/trimmed_%s.fa" % barcoder)
trimminfo = pd.read_csv(triminfo_fh, sep = "\t", index_col = 0,header=None,names=["start","end","strand","alignlength"])
record_dict = SeqIO.index(fasta_fh, "fasta")

In [20]:
#!/usr/bin/env python3
#Data 2019/07/15
#Auther: Qiutao DING, PhD candicate
#Department of Biology, Hong Kong Baptist University, Hong Kong, China
#Contact: qiutaod@gmail.com 
import os
import pandas as pd
from Bio import SeqIO
import argparse

def fileName(j=3):
    if int(j) <10:
        i = "barcode0"+str(j)
    else:
        i = "barcode"+str(j)
    return(i)

def trimming(barcoder):
    triminfo_fh = str("blastn_backbone_info_filtered/%s.csv" % barcoder)
    fasta_fh = str("read_with_backbone/%s.fa" % barcoder)
    output_fh = str("read_trimmed/trimmed_%s.fa" % barcoder)
    trimminfo = pd.read_csv(triminfo_fh, sep = "\t", index_col = 0,header=None,names=["start","end","strand","alignlength"])
    record_dict = SeqIO.index(fasta_fh, "fasta")
    with open(output_fh,"w") as fw:
        for readid in trimminfo.index:
            readinfo = trimminfo.loc[readid,]
            qstart = readinfo.start
            qstop = readinfo.end
            qstrand = readinfo.strand
            read = record_dict[readid]
            seq_L = ""
            seq_R = ""
            if qstrand == "+":
                if len(read) - qstop >= 100 and qstart >= 100:
                    seq_L = str(read.seq)[:qstart-1]
                    header_L = str(">%s_L len:%i strand:%s\n" % (read.id, len(seq_L), qstrand))
                    fw.write(header_L)
                    fw.write(seq_L+"\n")
                    seq_R = str(read.seq)[qstop :]
                    header_R = str(">%s_R len:%i strand:%s\n" % (read.id, len(seq_R), qstrand))
                    fw.write(header_R)
                    fw.write(seq_R+"\n")
                elif len(read) - qstop >= 100: 
                    # right flanking only
                    seq_R = str(read.seq)[qstop :]
                    header_R = str(">%s_R len:%i strand:%s\n" % (read.id, len(seq_R), qstrand))
                    fw.write(header_R)
                    fw.write(seq_R+"\n")
                elif qstart >= 100: 
                    # left flanking only
                    seq_L = str(read.seq)[:qstart-1]
                    header_L = str(">%s_L len:%i strand:%s\n" % (read.id, len(seq_L), qstrand))
                    fw.write(header_L)
                    fw.write(seq_L+"\n")
            elif qstrand == "-":
                if len(read) - qstop >= 100 and qstart >= 100: 
                    seq_L = str(read.seq[qstop:].reverse_complement())
                    seq_R = str(read.seq[:qstart-1].reverse_complement())
                    header_L = str(">%s_L len:%i strand:%s\n" % (read.id, len(seq_L), qstrand))
                    fw.write(header_L)
                    fw.write(seq_L+"\n")
                    header_R = str(">%s_R len:%i strand:%s\n" % (read.id, len(seq_R), qstrand))
                    fw.write(header_R)
                    fw.write(seq_R+"\n")
                elif len(read) - qstop >= 100: 
                    seq_L = str(read.seq[qstop:].reverse_complement())
                    header_L = str(">%s_L len:%i strand:%s\n" % (read.id, len(seq_L), qstrand))
                    fw.write(header_L)
                    fw.write(seq_L+"\n")
                elif qstart >= 100: 
                    seq_R = str(read.seq[:qstart-1].reverse_complement())
                    header_R = str(">%s_R len:%i strand:%s\n" % (read.id, len(seq_R), qstrand))
                    fw.write(header_R)
                    fw.write(seq_R+"\n") 
    fw.close()
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description = """
    will be added in the furture
    """)
    parser.add_argument("-b", "--barcode", help = "barcode number")
    args = parser.parse_args()
    #function calling
    trimming(fileName(args.barcode))
    print("Finished",fileName(args.barcode))

44b73770-d1c6-40f4-979b-8e069e21c2a8
f6660d39-eabb-4893-8b3f-cd61adbd03e0
c5c089c6-8f65-41ba-a0d1-145ae651ac55
b2fda24f-ef82-4b4e-a6ec-279b11aed47b
3ded4e78-ba29-4107-8a20-6a753122022d
